# Skip-Gram

::::{note}

このノートは [Skip-Gram実装課題](./skipgram.myst.md) のヒントになるように書かれています．

::::

CBoWと遂になる単語埋め込みベクトル作成手法である __Skip-Gram__ を実装します．ここでは，Negative Samplingのような技術を使わず，出力層でsoftmax関数を利用することで実装を簡単にしています．そのため計算コストが膨大になる傾向があり，大規模なコーパスに適用することはお勧めしません．

Skip-Gramの計算コストの大きさには出力層のSoftmax活性化関数が大きな影響を与えます．そのため，高速化を行うためにはSoftmaxを __Negative Sampling__ と呼ばれるアルゴリズムで代用することになります．これについては[このブログ](https://rf00.hatenablog.com/entry/2019/03/17/112317)が実装の助けになります．また，直接Skip-Gramを紹介しているわけではないのですが，CBoWの説明の中でこれを説明している[ゼロから作るDeep Learning ❷ ―自然言語処理編](https://www.amazon.co.jp/%E3%82%BC%E3%83%AD%E3%81%8B%E3%82%89%E4%BD%9C%E3%82%8BDeep-Learning-%E2%80%95%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86%E7%B7%A8-%E6%96%8E%E8%97%A4-%E5%BA%B7%E6%AF%85/dp/4873118360)も非常に参考になるでしょう．

In [1]:
# packageのimport
import re
import math 
from typing import Any
from tqdm.std import trange,tqdm
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy.sparse import lil_matrix

# pytorch関連のimport
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
import skorch
from skorch import NeuralNetClassifier, NeuralNetRegressor
from skorch.callbacks import Callback, EpochScoring
from torch.utils.data import Dataset

from janome.tokenizer import Tokenizer

## データの読み込み

コーパスにはja.text8のサブセットを利用します．発展課題に取り組む場合も，指定されているハイパーパラメータやコーパスのサイズが実行困難である場合は適宜修正してください．ただし，その場合はskip_gram.pyの先頭行に，docstringを用意してその旨を書いてください．あるいはCLIのオプションにしてもいいかもしれません．

In [3]:
with open("./data/ja.text8") as f:
    text8 = f.read()
print(text8[:200])

#LIMIT = math.floor(len(text8)*0.1)
LIMIT = 100_0000
print(f"{LIMIT}/ {len(text8)}")
text8 = text8[:LIMIT]

ちょん 掛け （ ちょん がけ 、 丁 斧 掛け ・ 手斧 掛け と も 表記 ） と は 、 相撲 の 決まり 手 の ひとつ で ある 。 自分 の 右 （ 左 ） 足 の 踵 を 相手 の 右 （ 左 ） 足 の 踵 に 掛け 、 後方 に 捻っ て 倒す 技 。 手斧 （ ちょう な ） を かける 仕草 に 似 て いる こと から 、 ちょう な が 訛っ て ちょん 掛け と なっ 
1000000/ 46507793


## 形態素解析

コーパス内の単語（トークン）全てを利用すると語彙が多くなりすぎるので，ここでは名詞（それも一般名詞と固有名詞）のみを利用します．そのために形態素解析を行う必要があるので，python製の形態素解析器であるjanomeを利用しています．形態素解析器はこれ以外にもMecabなどが有名です．

### 形態素解析器 janome

ja.text8の一部に品詞分解を行なった結果を以下に示します．


::::{margin}
:::{warning}
ja.text8は予め分かち書きされているため，以下の処理が正しく動作している保証はありません．
:::
::::

In [28]:
t = Tokenizer()
sample_text = "".join(text8[:50].split())
for token in t.tokenize(sample_text):
    print(token.surface, "\t", token.part_of_speech.split(","))

ちょん 	 ['名詞', '一般', '*', '*']
掛け 	 ['名詞', '接尾', '一般', '*']
（ 	 ['記号', '括弧開', '*', '*']
ちょん 	 ['名詞', '一般', '*', '*']
がけ 	 ['名詞', '接尾', '一般', '*']
、 	 ['記号', '読点', '*', '*']
丁 	 ['名詞', '固有名詞', '人名', '姓']
斧 	 ['名詞', '一般', '*', '*']
掛け 	 ['名詞', '接尾', '一般', '*']
・ 	 ['記号', '一般', '*', '*']
手斧 	 ['名詞', '一般', '*', '*']
掛け 	 ['名詞', '接尾', '一般', '*']
と 	 ['助詞', '格助詞', '引用', '*']
も 	 ['助詞', '係助詞', '*', '*']
表記 	 ['名詞', 'サ変接続', '*', '*']
） 	 ['記号', '括弧閉', '*', '*']
と 	 ['助詞', '格助詞', '引用', '*']
は 	 ['助詞', '係助詞', '*', '*']
、 	 ['記号', '読点', '*', '*']
相撲 	 ['名詞', '一般', '*', '*']


### janomeを使った語彙辞書作成

活用する語彙をまとめた辞書（word2id, id2word）を作成します．この実装はダーティなので，実際に自然言語処理を行う場合は参考にしないでください．

In [5]:
def my_analyzer(text):
    #text = code_regex.sub('', text)
    #tokens = text.split()
    #tokens = filter(lambda token: re.search(r'[ぁ-ん]+|[ァ-ヴー]+|[一-龠]+', token), tokens)
    tokens = []
    for token in tqdm(t.tokenize(text)):
        pos = token.part_of_speech.split(",")
        if "名詞" == pos[0]:
            if "一般" == pos[1] or "固有名詞" == pos[1]:
                tokens.append(token.surface)
    tokens = filter(lambda token: re.search(r'[ぁ-ん]+|[ァ-ヴー]+|[一-龠]+', token), tokens)
    return tokens 

def build_contexts_and_target(corpus, window_size:int=5)->tuple[np.ndarray,np.ndarray]:
    contexts = []
    target = []
    vocab = set()
    _window_size = window_size//2
    # 文ごとに分割
    preprocessed_corpus = corpus.replace(" ","")
    # posを見て単語ごとに分割
    tokens = list(my_analyzer(preprocessed_corpus))

    # 新しい語彙を追加
    vocab = vocab | set(tokens)

    # スライディングウィンドウ
    for i in trange(_window_size, len(tokens)-_window_size):
        # ウィンドウの真ん中をtargetにする
        target.append(tokens[i])
        # 真ん中以外の単語をcontextsへ
        tmp = tokens[i-_window_size:i]
        tmp += tokens[i+1:i+1+_window_size]
        contexts.append(tmp)

    # 辞書作成
    id2word = list(vocab)
    word2id = {word:id for id,word in enumerate(id2word)}
    vocab_size = len(word2id)


    # contextsとtargetを単語id配列へ置き換え
    contexts_id_list = [[word2id[word] for word in doc] for doc in contexts]
    target_id_list = [word2id[word] for word in target]


    contexts = lil_matrix((len(contexts_id_list), vocab_size),dtype=np.float32)
    for index, _contexts_id_list in enumerate(contexts_id_list):
        #tmp = np.eye(vocab_size)[np.array(_contexts_id_list)]
        for word_id in _contexts_id_list:
            contexts[index, word_id] +=1.

    target = np.array(target_id_list)
    return contexts.tocsr().astype(np.float32), target, word2id, id2word

WINDOW_SIZE = 11
contexts, target, word2id, id2word = build_contexts_and_target(text8, window_size=WINDOW_SIZE)
print(f"contextsのshape: {contexts.shape}")

363003it [00:18, 19741.90it/s]
100%|██████████| 80264/80264 [00:00<00:00, 816091.50it/s]


contextsのshape: (80264, 17871)


## クラスの作成

Skip-gramをnn.Moduleのサブクラスとして実装します．

![](https://cdn-ak.f.st-hatena.com/images/fotolife/r/rf00/20190316/20190316165423.png)

クラスの実装には上のskip-gramアーキテクチャ図を参考にしてください．高速化のテクニックなどは不要です．（もちろん実装できる人は実装してもOK）

In [25]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size:int, embedding_dim:int)->None:
        super().__init__()
        ...

    def forward(self, input:torch.Tensor)->torch.Tensor:
        ...

## 損失関数の作成

Skip-Gramはクラス分類の体裁をとっているので，損失関数にはCross Entropyを用います．ただしPyTorchで用意されているnn.CrossEntropyを用いることは（おそらく）できないので，自作しましょう．

:::{hint}

条件：
- batch_size=128, vocab_size=11342のとき，以下が損失関数に入力されると仮定して実装してください．  
    - SkipGramがforwardメソッドから出力するtensor．shapeは「torch.Size([128, 11342])」，
    - 正解データとして利用するtensor．shapeは「torch.Size([128, 11342])」
- callbackにおいて，ここで実装したcross entropyを使ってperplexityを計算します．


In [27]:
class BowCrossEntropy(nn.Module):
    def forward(self, input, target):
        """
        inputはSkip-gramの出力です．
        targetは予測したいcontextsです．
        """
        ...

## trainerの準備と訓練


ここまでの実装が終わったら，あとは訓練用のプログラムを書くだけです．この解説ではskorchを利用して楽をします．Skip-Gramはクラス分類の体裁を取っていると言いましたが，出力はcategoricalではなくmultinomialです．つまり __一つのデータに対して正解ラベルが複数あります__ ．これはskorchの`NeuralNetClassifier`では上手く扱えないので，`NeuralNetRegressor` を使っています．

:::{note}
- NeuralNetClassifierは主に1データ1ラベルの場合に利用します．今回の例でも使えないわけではないのですが，標準で設定された「正答率を表示するコールバック」が動作してしまうので利用を見送りました．
- `EpochScoring(lambda net,X=None,y=None: np.exp(net.history_[-1, "valid_loss"]), name="valid_ppl"), `はエポックの終わりに呼び出されるコールバック関数の雛形である`EpochScoring`を利用して，Perplexityを計算します．
- targetもcontextsもnp.ndarrayのままでfitに渡します．
    - trainerが中でdatasetやdataloaderを用意してくれます．
    - contextsはscipy.sparse.lil_matrix or scipy.sparse.csr_matrixになっているので，`toarray`メソッドでnp.ndarrayに戻しています．
:::

:::{margin}

今回は実装の簡単さを優先したので，メモリ効率が非常に悪い実装になっていることに注意してください．RAM 16GB程度あれば動作するはずです．

contexts配列のshapeが(80264, 17871)であり，dtype=float32である場合，
```python
import sys 
K = 1024
M = K**2
print(f"csr_matrix: {sys.getsizeof(contexts)} B")
print(f"np.ndarray: {sys.getsizeof(contexts.toarray())/M} MB")
```
contexts変数に紐づくオブジェクトの使用メモリは以下の通り：
```
csr_matrix: 48 B
np.ndarray: 5471.794036865234 MB
```
通常の配列で保持すると，shape[0]*shape[1]に比例してメモリを消費します．BoWのような要素がほぼほぼ0で一部が0以外の行列を疎行列と呼びますが，疎行列に特化した型であるcsr_matrixやlil_matrixを使うと，0以外の要素の値とインデックスのみを保持する設計になっているのでメモリ消費量が劇的に減ります．

:::

In [8]:
trainer = NeuralNetRegressor(
    SkipGram(len(word2id), 50),
    optimizer=optim.Adam,
    criterion=BowCrossEntropy,
    max_epochs=20,
    batch_size=128,
    lr=0.01,
    callbacks=[
        EpochScoring(lambda net,X=None,y=None: np.exp(net.history_[-1, "valid_loss"]), name="valid_ppl"), 
        EpochScoring(lambda net,X=None,y=None: np.exp(net.history_[-1, "train_loss"]), name="train_ppl", on_train=True,)
    ],
    device="cpu", # 適宜変更
)

trainer.fit(target, contexts.toarray())

  epoch    train_loss    train_ppl    valid_loss    valid_ppl      dur
-------  ------------  -----------  ------------  -----------  -------
      1        9.5161   13577.1923        9.5077   13463.3985  10.8592
      2        8.6090    5480.6118        9.5838   14527.7893  9.1960
      3        7.9813    2925.5963        9.7863   17787.6748  9.1649
      4        7.5530    1906.4801        9.9466   20880.6048  9.2059
      5        7.2634    1427.0374       10.1053   24473.2919  9.4628
      6        7.0630    1167.9362       10.2429   28083.5366  10.0126
      7        6.9139    1006.1204       10.3765   32096.6788  10.0011
      8        6.8048     902.1609       10.4951   36139.6945  10.0044
      9        6.7210     829.6149       10.6081   40461.6903  9.7140
     10        6.6579     778.9301       10.7094   44773.3498  9.1142
     11        6.6072     740.4165       10.8048   49255.9870  9.2038
     12        6.5677     711.7535       10.8919   53738.0492  9.6758
     13       

<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=SkipGram(
    (embedding): Embedding(17871, 50, max_norm=1)
    (linear): Linear(in_features=50, out_features=17871, bias=True)
  ),
)

## 類似単語検索

cbowと同様に単語埋め込みベクトルを使って，類似単語の検索を行います．

In [9]:
def get_similar_words(query, word_embeddings, topn=5, word2id=word2id, ):
    """単語埋め込みベクトルを使って似た単語を検索する

    Args:
        query (str): 類似単語を検索したい単語
        topn (int, optional): 検索結果の表示個数. Defaults to 5.
        word2id (dict[str,int], optional): 単語→単語idの辞書. Defaults to word2id.
        word_embeddings (np.ndarray, optional): 単語埋め込み行列．必ず(語彙数x埋め込み次元数)の行列であること. Defaults to word_embeddings.
    """
    id=word2id[query]
    E = (word_embeddings.T / np.linalg.norm(word_embeddings,ord=2, axis=1)).T # {(V,L).T / (V)}.T = (V,L)
    target_vector = E[id]
    cossim = E @ target_vector # (V,L)@(L)=(V)
    sorted_index = np.argsort(cossim)[::-1][1:topn+1] # 最も似たベクトルは自分自身なので先頭を除外

    print(f">>> {query}")
    _id2word = list(word2id.keys())
    for rank, i in enumerate(sorted_index):
        print(f"{rank+1}:{_id2word[i]} \t{cossim[i]}")

word_embeddings = trainer.module_.embedding.weight.detach().cpu().numpy()

get_similar_words("ロボット", word_embeddings, )

>>> ロボット
1:ユニバーサル 	0.898608386516571
2:ポルト 	0.7893995642662048
3:ロボティックス 	0.763614296913147
4:テラ 	0.742680013179779
5:関節 	0.7259170413017273


In [11]:
get_similar_words("サッカー", word_embeddings, )
get_similar_words("日本", word_embeddings, )
get_similar_words("女王", word_embeddings, )
get_similar_words("機械学習", word_embeddings, )

>>> サッカー
1:リーグ 	0.734089195728302
2:専業 	0.7245967388153076
3:ヴァンフォーレ 	0.6850863695144653
4:選手 	0.6845436692237854
5:アルビレックス 	0.6741206645965576
>>> 日本
1:ほん 	0.6705817580223083
2:米国 	0.6255179047584534
3:王者 	0.6063108444213867
4:社団 	0.5765134692192078
5:蓄音機 	0.5684884786605835
>>> 女王
1:ヴィクトリアシリーズ 	0.6750556826591492
2:後塵 	0.649889349937439
3:ティアラカップ 	0.641579806804657
4:ボウラー 	0.6231715083122253
5:シェクター 	0.6060587763786316


KeyError: '機械学習'

今回の解説ではja.text8のサブセットを利用しているせいで，この単語埋め込みがカバーしている語彙に「機械学習」は含まれていないようです．